In [1]:
import skimage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

# packages used for this script specifically
from skimage import filters
from skimage import exposure
import mahotas as mh
from skimage.morphology import disk, binary_erosion, binary_dilation, binary_opening, binary_closing

from itertools import compress

This script calculates the localized mucus-bacteria ratio based on the MIBI imaging data.

In [2]:
# some helper function: # define a quick function for sliding windon

def sliding_window(image, stepSize, windowSize):
    for y in range(50, image.shape[0], stepSize):
        for x in range(50, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])
            
# define action function:
def extractFeatures(window):
    total = window[2].shape[0] * window[2].shape[1]
    sigsum = np.sum(window[2])
    return sigsum

In [3]:
# create csv of single cell extracted


csv_out = '../../../final_data/bac_csv/'
img_dir = '../../../final_data/normed/micro_fine1ms/'
img_dir_mask = '../../../final_data/masked/micro_fine1ms/'


#runs = ['2023-03-04T16-22-15-DSSC1-Run1']

runs = ['2023-03-04T16-22-15-DSSC1-Run1', '2023-03-04T19-46-22-DSSC1-Run2',
             '2023-03-04T23-36-59-DSSC1-Run3', '2023-03-05T15-51-42-DSST2-Run1',
             '2023-03-05T18-22-42-DSST2-Run2', '2023-03-05T21-42-35-DSST2-Run3',
             '2023-03-05T23-58-06-DSST2-Run4', '2023-03-06T11-23-28-DSST2-Run5',
             '2023-03-06T13-40-46-DSST2-Run6', '2023-03-06T16-15-43-DSSC2-Run1',
             '2023-03-07T18-32-49-DSSC2-Run5', '2023-03-07T16-08-39-DSSC2-Run4',
             '2023-03-07T13-54-48-DSSC2-Run3', '2023-03-07T11-17-56-DSSC2-Run2',
             '2023-03-09T15-53-36-DSSC3-Run4', '2023-03-09T13-34-52-DSSC3-Run3',
             '2023-03-09T10-32-36-DSSC3-Run2', '2023-03-08T23-51-06-DSSC3-Run1',
             '2023-03-08T21-26-09-DSST3-Run6', '2023-03-08T18-48-43-DSST3-Run5',
             '2023-03-08T16-15-19-DSST3-Run4', '2023-03-08T14-03-50-DSST3-Run3',
             '2023-03-08T11-39-28-DSST3-Run2', '2023-03-07T22-15-40-DSST3-Run1',
            '2023-03-10T12-31-44-DSST4-Run1', '2023-03-10T14-47-38-DSST4-Run2', '2023-03-10T17-34-14-DSST4-Run3',
            '2023-03-18T14-36-10-DSST4-Run4', '2023-03-18T17-28-03-DSST4-Run5', '2023-03-18T23-11-36-DSST4-Run6',
            '2023-03-19T12-45-44-DSSC4-Run1', '2023-03-19T15-20-40-DSSC4-Run2', '2023-03-19T17-38-15-DSSC4-Run3',
            '2023-03-19T20-32-23-DSSC4-Run4', '2023-03-20T12-07-46-DSST1-Run1', '2023-03-20T14-32-22-DSST1-Run2',
            '2023-03-20T19-13-53-DSST1-Run3', '2023-03-20T21-44-51-DSST1-Run4', '2023-03-21T24-29-43-DSST1-Run5',
            '2023-03-21T10-32-42-DSST1-Run6', '2023-03-21T12-44-51-DSST1-Run7', '2023-03-28T11-45-15-DSSC1-Run4'
       ]

####### start running

fovll = []
runll = []
rall = []
rmll = []

for run in runs:
    print("working on ", run)
    path_masked = os.path.join(img_dir, run)
    fovs = os.listdir(path_masked)
    
    for fov in fovs:
        
        if fov == '.DS_Store':
            continue
        
        if os.path.exists(img_dir_mask + run + '/' + fov + '/Mask.tif') == False: # fov without mask no bacteria
            continue
        
        ## read in the nec images
        muc = skimage.io.imread( img_dir + run + '/' + fov + '/Muc2.tiff')
        alx488 = skimage.io.imread( img_dir + run + '/' + fov + '/Alx488_v3.tiff')
        mask = skimage.io.imread( img_dir_mask + run + '/' + fov + '/Mask.tif')
        
        # preprocess that binarize
        q1 = np.quantile(muc, 0.03)
        q2 = np.quantile(alx488, 0.001)
        muc[muc<q1] = q1
        alx488[alx488<q2] = q2
        
        # binarize
        muc_b = muc.copy()
        alx488_b = alx488.copy()
        alx488_b[alx488_b>=0.015] = 1 # these are hard coded values that has been manually evaluated
        alx488_b[alx488_b<0.015] = 0
        muc_b[muc_b>=0.05] = 1
        muc_b[muc_b<0.05] = 0
        muc_b[mask != 0] = 0
        
        ########## start calculating correlations
        
        ### for bacteria
        windows_bac = sliding_window(alx488_b, 10, (100, 100)) # slide step 1, window size 100 pix
        perc_list_bac = []

        for window in windows_bac: # bac size
            featureVector = extractFeatures(window)
            perc_list_bac.append(featureVector)
    
        ### for mucus
        windows_muc = sliding_window(muc_b, 10, (100, 100)) # slide step 1, window size 100 pix
        perc_list_muc = []

        for window in windows_muc: # muc size
            featureVector = extractFeatures(window)
            perc_list_muc.append(featureVector)
            
        ### identify host areas and remove from analysis
        mask_temp = mask.copy()
        mask_temp[mask!=0] = 1
        windows_mask = sliding_window(mask_temp, 10, (100, 100)) # slide step 1, window size 100 pix
        perc_list_mask = []

        for window in windows_mask:
            featureVector = extractFeatures(window)
            nonmaskzie = 100 * 100 - featureVector
            perc_list_mask.append(nonmaskzie)
            
        ### calculate sliding window ratio
        # index to take
        ratiolist = []
        
        for i in range(len(perc_list_mask)):
            
            if perc_list_mask[i] <= 5000:
                continue
            else:
                ratio = ((perc_list_muc[i]+1)/perc_list_mask[i]) / ((perc_list_bac[i]+1)/perc_list_mask[i])
                ratiolist.append(ratio)
        
        ratioavg = np.mean(ratiolist)
        ratiomed = np.median(ratiolist)
        #print(np.var(ratiolist))
        #print(np.median(ratiolist))
        
        ### save the results
        fovll.append(fov)
        runll.append(run)
        rall.append(ratioavg)
        rmll.append(ratiomed)
        

working on  2023-03-04T16-22-15-DSSC1-Run1
working on  2023-03-04T19-46-22-DSSC1-Run2
working on  2023-03-04T23-36-59-DSSC1-Run3
working on  2023-03-05T15-51-42-DSST2-Run1
working on  2023-03-05T18-22-42-DSST2-Run2
working on  2023-03-05T21-42-35-DSST2-Run3
working on  2023-03-05T23-58-06-DSST2-Run4
working on  2023-03-06T11-23-28-DSST2-Run5
working on  2023-03-06T13-40-46-DSST2-Run6
working on  2023-03-06T16-15-43-DSSC2-Run1
working on  2023-03-07T18-32-49-DSSC2-Run5
working on  2023-03-07T16-08-39-DSSC2-Run4
working on  2023-03-07T13-54-48-DSSC2-Run3
working on  2023-03-07T11-17-56-DSSC2-Run2
working on  2023-03-09T15-53-36-DSSC3-Run4
working on  2023-03-09T13-34-52-DSSC3-Run3
working on  2023-03-09T10-32-36-DSSC3-Run2
working on  2023-03-08T23-51-06-DSSC3-Run1
working on  2023-03-08T21-26-09-DSST3-Run6
working on  2023-03-08T18-48-43-DSST3-Run5
working on  2023-03-08T16-15-19-DSST3-Run4
working on  2023-03-08T14-03-50-DSST3-Run3
working on  2023-03-08T11-39-28-DSST3-Run2
working on 

In [4]:
dd = {'fov':fovll, 'run':runll, 'rall':rall, 'rmll':rmll}

data = pd.DataFrame.from_dict(dd)
data

,fov,run,rall,rmll
0,fov-2-scan-1,2023-03-04T16-22-15-DSSC1-Run1,2.388976,1.126126
1,fov-4-scan-1,2023-03-04T16-22-15-DSSC1-Run1,3.312774,0.250000
2,fov-3-scan-1,2023-03-04T16-22-15-DSSC1-Run1,1.060751,0.080000
3,fov-1-scan-1,2023-03-04T16-22-15-DSSC1-Run1,1.421733,0.222222
4,fov-5-scan-1,2023-03-04T16-22-15-DSSC1-Run1,0.361976,0.036541
...,...,...,...,...
186,fov-4-scan-1,2023-03-28T11-45-15-DSSC1-Run4,21.920009,1.193638
187,fov-6-scan-1,2023-03-28T11-45-15-DSSC1-Run4,3.193677,0.166667
188,fov-3-scan-1,2023-03-28T11-45-15-DSSC1-Run4,62.590546,1.000000
189,fov-1-scan-1,2023-03-28T11-45-15-DSSC1-Run4,13.538955,0.061224


In [58]:
data.to_csv('../../../final_data/bac_csv/nbhd_ratioV1.csv')

Plotting etc done in seperate R script